In [4]:
import pandas as pd
import numpy as np
import os
import emoji
import nltk
import re
import string
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [5]:
# Modeling
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [7]:
data = pd.read_csv('data10.csv')

In [ ]:
data.head()

,writer,doc,is_irony
0,88d4d2c7da1c10784ab578fd7b3725c5,"['""Nearly 1 in 3 LGBTQ people live in the Sout...",0
1,19af9106baf2208c9eef017ef6002996,['#USER# Chloe\'s done amazing things for tran...,0
2,84c6db87916b5de77e53444c1e3a15be,"['#USER# You, as ladies, were actually sown in...",1
3,2b3a0410dba9d6a04e903f459e511648,['#USER# #USER# I was also hoping he\'d run th...,0
4,6782740a7a7210777f6b76835219e0d3,['#USER# How many likes do you need for us to...,0


In [8]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
def text_clean(text):
    text = text.lower()
    # clean the 
    emoji = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text = emoji.sub(r'', text)
    text = text.lower()
    text = re.sub(r"hashtag", "", text)
    text = re.sub(r"url", "", text)
    text = re.sub(r"user", "", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)        
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text) 
    text = re.sub(r"\'ll", " will", text)  
    text = re.sub(r"\'ve", " have", text)  
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text)
    return text

In [10]:
def CleanTokenize(df):
    doc_processed = list()
    lines = df["doc"].values.tolist()

    for line in lines:
        line = text_clean(line)
        # tokenize the text
        tokens = word_tokenize(line)
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        words = [w for w in words if not w in stop_words]
        doc_processed.append(words)
    return doc_processed

In [12]:
doc_processed = CleanTokenize(data)
# doc_processed[0:10]

Embedding texts_to_sequences

In [ ]:
train_split = 0.8
# val_split = 0.2
max_length = 10000

# tokenize
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(doc_processed)
sequences = tokenizer_obj.texts_to_sequences(doc_processed)

word_index = tokenizer_obj.word_index
print("unique tokens - ",len(word_index))
vocab_size = len(tokenizer_obj.word_index) + 1
print('vocab size -', vocab_size)

lines_pad = pad_sequences(sequences, maxlen=max_length, padding='post')
sentiment =  data['is_irony'].values

indices = np.arange(lines_pad.shape[0])
np.random.shuffle(indices)
lines_pad = lines_pad[indices]
sentiment = sentiment[indices]

train_samples = int(train_split * lines_pad.shape[0])
# test_samples = int(lines_pad.shape[0] * (train_split + val_split))

X_train = lines_pad[:train_samples]
y_train = sentiment[:train_samples]
X_test = lines_pad[train_samples:]
y_test = sentiment[train_samples:]

unique tokens -  102746
vocab size - 102747


Random Forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=500,random_state=2022)
rfc.fit(X_train_pad,y_train)
preds = rfc.predict(X_test_pad)
print('accuracy: ', accuracy_score(y_test,preds))
print(classification_report(y_test,preds))

accuracy:  0.4642857142857143
              precision    recall  f1-score   support

           0       0.49      0.39      0.43        44
           1       0.45      0.55      0.49        40

    accuracy                           0.46        84
   macro avg       0.47      0.47      0.46        84
weighted avg       0.47      0.46      0.46        84



In [ ]:
rf_seq = RandomForestClassifier(random_state=1)
rf_seq.fit(X_train, y_train)
y_hat_test = rf_seq.predict(X_test)
rf_seq_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
rf_seq_test_acc

0.8

LinearSVC Classifier

In [ ]:
# Instantiate classifier, fit, and predict on test data

svm_seq = LinearSVC(random_state=1, max_iter=10000)
svm_seq.fit(X_train, y_train)
y_hat_test = svm_seq.predict(X_test)
svm_seq_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
svm_seq_test_acc

0.62

Multinomial Naive Bayes Classifier

In [ ]:
scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
mnb_seq_clf = MultinomialNB()
mnb_seq_clf.fit(X_train_sc, y_train)
y_hat_test = mnb_seq_clf.predict(X_test_sc)

In [ ]:
mnb_seq_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
mnb_seq_test_acc

0.77

AdaBoost Classifier

In [ ]:
ada_seq_clf = AdaBoostClassifier(random_state=1)
ada_seq_clf.fit(X_train, y_train)
y_hat_test = ada_seq_clf.predict(X_test)
ada_seq_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
ada_seq_test_acc

0.69

XGBoost Classifier

In [ ]:
xgb_seq_clf = XGBClassifier(random_state=1)
xgb_seq_clf.fit(X_train, y_train)
y_hat_test = xgb_seq_clf.predict(X_test)

xgb_seq_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
xgb_seq_test_acc

0.76

Neural Network

In [ ]:

model = Sequential()
model.add(Dense(100, input_dim=10000, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile model

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

# Fit to training data
model.fit(X_train, y_train, epochs=3, batch_size=100, 
          validation_data=(X_test, y_test))

Epoch 1/3
4/4 [==============================] - 1s 70ms/step - loss: 29478.9648 - accuracy: 0.5179 - val_loss: 3679.1216 - val_accuracy: 0.5357
Epoch 2/3
4/4 [==============================] - 0s 28ms/step - loss: 1207.7065 - accuracy: 0.6577 - val_loss: 588.7953 - val_accuracy: 0.5952
Epoch 3/3
4/4 [==============================] - 0s 28ms/step - loss: 179.8202 - accuracy: 0.5149 - val_loss: 92.2848 - val_accuracy: 0.5476


In [ ]:
#Predictions
y_hat_test = np.argmax(model.predict(X_test), axis=-1)
nn_seq_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
nn_seq_test_acc

0.74

Summary

In [ ]:
# Create classifier summary table

models = ['Linear SVC', 'Random Forest', 'Multinomial Naive Bayes', 
                  'AdaBoost', 'XGBoost', 'Neural Network']

seq_model_accs = [svm_seq_test_acc, rf_seq_test_acc, mnb_seq_test_acc, 
                    ada_seq_test_acc, xgb_seq_test_acc, nn_seq_test_acc]

seq_model_summary = pd.DataFrame([models, seq_model_accs]).T

seq_model_summary.columns = ['Classifier', 'sequence Accuracy']

seq_model_summary

,Classifier,sequence Accuracy
0,Linear SVC,0.62
1,Random Forest,0.8
2,Multinomial Naive Bayes,0.77
3,AdaBoost,0.69
4,XGBoost,0.76
5,Neural Network,0.74


Embedding TF-IDF & Random Forest

In [93]:
tokenlist=[]
for i in range(len(doc_processed)):
    a=' '.join(doc_processed[i])
    tokenlist.append(a)

tokenlist[0] 

'nearly lgbtq people live south see shit shitting queer black man guise antisouthern great sincerity say go fuck n ok throwback susan b anthony reallysomethingjust okay den okay yall got enjoy anyway anyone articlesresources nonbinary folks innavigating combat arts please send waynnesp afab binary folks martial arts got ta believe people show us casteism folks dont talk enough shapes tech esp moments like oh way mark employees gon na burn hell thats convo black natives lead though antiblackness deadly impacts behind black people designed target got hip indigenous folks stories aliens seeing stories scattered across globeyeah methings need get better going map someone asks im big ass shoutout trans nonbinary folks followingnni wouldnt made without yall corner even knew ppl sounding alarm emergent strain fascist feminism since first wave yeah yeah brand new emerging everyone soooooo shocked food plus though awhile since jollof rice goodyeahhhhh yall dont want love yall want someone whose

In [94]:
tfIdfVec = TfidfVectorizer(max_features=1000)
tfIdfVec.fit(tokenlist)

TfidfVectorizer(max_features=1000)

In [95]:
tf_idf_x = tfIdfVec.transform(tokenlist)

In [96]:
y = data['is_irony'].values

X_train, X_test, y_train, y_test = train_test_split(tf_idf_x.todense(),y,test_size=0.2,random_state=1)

Random Forest

In [46]:
rfc = RandomForestClassifier(n_estimators=500,random_state=42)
rfc.fit(X_train, y_train)
preds = rfc.predict(X_test)
print('accuracy: ', accuracy_score(y_test, preds))
print(classification_report(y_test, preds))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


accuracy:  0.8928571428571429
              precision    recall  f1-score   support

           0       0.95      0.84      0.89        43
           1       0.85      0.95      0.90        41

    accuracy                           0.89        84
   macro avg       0.90      0.89      0.89        84
weighted avg       0.90      0.89      0.89        84



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [49]:
rf_tfidf = RandomForestClassifier(n_estimators=100, criterion='entropy',random_state=1)
rf_tfidf.fit(X_train, y_train)
y_hat_test = rf_tfidf.predict(X_test)
rf_tfidf_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
rf_tfidf_test_acc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.9

LinearSVC Classifier

In [54]:
# Instantiate classifier, fit, and predict on test data

svm_tfidf = LinearSVC(random_state=1, max_iter=500)
svm_tfidf.fit(X_train, y_train)
y_hat_test = svm_tfidf.predict(X_test)
svm_tfidf_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
svm_tfidf_test_acc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.94

Multinomial Naive Bayes Classifier

In [ ]:
scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
mnb_tfidf_clf = MultinomialNB()
mnb_tfidf_clf.fit(X_train_sc, y_train)
y_hat_test = mnb_tfidf_clf.predict(X_test_sc)

In [ ]:
mnb_tfidf_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
mnb_tfidf_test_acc

0.89

AdaBoost Classifier

In [24]:
ada_tfidf_clf = AdaBoostClassifier(n_estimators=50, random_state=1)
ada_tfidf_clf.fit(X_train, y_train)
y_hat_test = ada_tfidf_clf.predict(X_test)
ada_tfidf_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
ada_tfidf_test_acc

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.87

XGBoost Classifier

In [ ]:
xgb_tfidf_clf = XGBClassifier(random_state=1)
xgb_tfidf_clf.fit(X_train, y_train)
y_hat_test = xgb_tfidf_clf.predict(X_test)

xgb_tfidf_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
xgb_tfidf_test_acc

0.92

Neural Network

In [98]:
model = Sequential()
model.add(Dense(1000, input_dim=1000, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile model

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

# Fit to training data
model.fit(X_train, y_train, epochs=9, batch_size=100, 
          validation_data=(X_test, y_test))

Epoch 1/9
4/4 [==============================] - 0s 44ms/step - loss: 0.6663 - accuracy: 0.6280 - val_loss: 0.5914 - val_accuracy: 0.8810
Epoch 2/9
4/4 [==============================] - 0s 9ms/step - loss: 0.5562 - accuracy: 0.8423 - val_loss: 0.4930 - val_accuracy: 0.8929
Epoch 3/9
4/4 [==============================] - 0s 10ms/step - loss: 0.4601 - accuracy: 0.8631 - val_loss: 0.4078 - val_accuracy: 0.9167
Epoch 4/9
4/4 [==============================] - 0s 9ms/step - loss: 0.3850 - accuracy: 0.8810 - val_loss: 0.3467 - val_accuracy: 0.9167
Epoch 5/9
4/4 [==============================] - 0s 10ms/step - loss: 0.3144 - accuracy: 0.8958 - val_loss: 0.2960 - val_accuracy: 0.8929
Epoch 6/9
4/4 [==============================] - 0s 10ms/step - loss: 0.2711 - accuracy: 0.9077 - val_loss: 0.2672 - val_accuracy: 0.9405
Epoch 7/9
4/4 [==============================] - 0s 9ms/step - loss: 0.2297 - accuracy: 0.9315 - val_loss: 0.2487 - val_accuracy: 0.9524
Epoch 8/9
4/4 [======================

In [43]:
#Predictions
y_hat_test = np.argmax(model.predict(X_test), axis=-1)
nn_tfidf_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
nn_tfidf_test_acc

0.95

Summary

In [ ]:
# Create classifier summary table

models = ['Linear SVC', 'Random Forest', 'Multinomial Naive Bayes', 
                  'AdaBoost', 'XGBoost', 'Neural Network']

tfidf_model_accs = [svm_tfidf_test_acc, rf_tfidf_test_acc, mnb_tfidf_test_acc, 
                    ada_tfidf_test_acc, xgb_tfidf_test_acc, nn_tfidf_test_acc]

tfidf_model_summary = pd.DataFrame([models, tfidf_model_accs]).T

tfidf_model_summary.columns = ['Classifier', 'TF-IDF Accuracy']

tfidf_model_summary

,Classifier,TF-IDF Accuracy
0,Linear SVC,0.94
1,Random Forest,0.92
2,Multinomial Naive Bayes,0.89
3,AdaBoost,0.88
4,XGBoost,0.92
5,Neural Network,0.9


Embedding GloVe

In [44]:
total_vocabulary = set(word for tweet in doc_processed for word in tweet)

len(total_vocabulary)
print('There are {} unique tokens in the dataset.'.format(
    len(total_vocabulary)))

There are 102746 unique tokens in the dataset.


In [45]:
glove = {}
with open(r'/content/drive/MyDrive/Language_processing/glove/glove.twitter.27B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [46]:
# Check length of glove dictionary relative to total words

print(f'{len(glove)} of {len(total_vocabulary)} \
    tokens in dataset matched in GloVe and vectorized!')

50508 of 102746     tokens in dataset matched in GloVe and vectorized!


In [47]:
class W2vVectorizer(object):
    """Class to generate mean word embeddings from word vectors. This class is
    quoted from Flatiron School Curriculum Learn.co Mod 4 appendix"""
    def __init__(self, w2v):
        # Takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note: Even though it doesn't do anything, 
    # it's required that this object implement a fit method or else
    # it can't be used in a scikit-learn pipeline  
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        # X should be a series of lists of tokens
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [48]:
vectorizer = W2vVectorizer(glove)

In [49]:
X_glove = vectorizer.transform(doc_processed)
y = data['is_irony'].values

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_glove, y, 
                                                    test_size=0.2, 
                                                    random_state=1)

Random Forest

In [66]:
rf_glove = RandomForestClassifier(n_estimators=200, criterion='gini',random_state=1)
rf_glove.fit(X_train, y_train)
y_hat_test = rf_glove.predict(X_test)

In [67]:
rf_glove_test_acc = round(accuracy_score(y_test, y_hat_test), 2)

In [68]:
rf_glove_test_acc

0.92

In [ ]:
print('accuracy: ', accuracy_score(y_test,y_hat_test))
print(classification_report(y_test,y_hat_test))

accuracy:  0.9047619047619048
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        43
           1       0.88      0.93      0.90        41

    accuracy                           0.90        84
   macro avg       0.91      0.91      0.90        84
weighted avg       0.91      0.90      0.90        84



LinearSVC Classifier

In [77]:
# Instantiate classifier, fit, and predict on test data

svm_glove = LinearSVC(random_state=1, max_iter=500)
svm_glove.fit(X_train, y_train)
y_hat_test = svm_glove.predict(X_test)

In [78]:
svm_glove_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
svm_glove_test_acc

0.89

Multinomial Naive Bayes Classifier 

In [ ]:
scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
mnb_glove_clf = MultinomialNB()
mnb_glove_clf.fit(X_train_sc, y_train)
y_hat_test = mnb_glove_clf.predict(X_test_sc)

In [ ]:
# Evaluate classifier

mnb_glove_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
mnb_glove_test_acc

0.64

AdaBoost Classifier

In [83]:
ada_glove_clf = AdaBoostClassifier(n_estimators=50, random_state=1)
ada_glove_clf.fit(X_train, y_train)
y_hat_test = ada_glove_clf.predict(X_test)

In [84]:
# Evaluate classifier

ada_glove_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
ada_glove_test_acc

0.86

XGBoost Classifier

In [ ]:
xgb_glove_clf = XGBClassifier(random_state=1)
xgb_glove_clf.fit(X_train, y_train)
y_hat_test = xgb_glove_clf.predict(X_test)

In [ ]:

xgb_glove_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
xgb_glove_test_acc

0.87

Neural Network LSTM

In [91]:
nn_glove = Sequential()
nn_glove.add(Dense(1000, input_dim=50, activation='relu'))
nn_glove.add(Dense(2, activation='softmax'))

# compile the keras model

nn_glove.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
                     metrics=['accuracy'])

# fit the keras model on the dataset
nn_glove.fit(X_train, y_train, epochs=20, batch_size=64, 
                 validation_data=(X_test, y_test))

# Predict on test data

y_hat_test = np.argmax(nn_glove.predict(X_test), axis=-1)

Epoch 1/20
6/6 [==============================] - 0s 29ms/step - loss: 0.6967 - accuracy: 0.5298 - val_loss: 0.6667 - val_accuracy: 0.5952
Epoch 2/20
6/6 [==============================] - 0s 7ms/step - loss: 0.6524 - accuracy: 0.5982 - val_loss: 0.6413 - val_accuracy: 0.6190
Epoch 3/20
6/6 [==============================] - 0s 7ms/step - loss: 0.6371 - accuracy: 0.6220 - val_loss: 0.6233 - val_accuracy: 0.5952
Epoch 4/20
6/6 [==============================] - 0s 6ms/step - loss: 0.6125 - accuracy: 0.6161 - val_loss: 0.6129 - val_accuracy: 0.6667
Epoch 5/20
6/6 [==============================] - 0s 6ms/step - loss: 0.6002 - accuracy: 0.6429 - val_loss: 0.5965 - val_accuracy: 0.6190
Epoch 6/20
6/6 [==============================] - 0s 7ms/step - loss: 0.5864 - accuracy: 0.6786 - val_loss: 0.5757 - val_accuracy: 0.6905
Epoch 7/20
6/6 [==============================] - 0s 6ms/step - loss: 0.5699 - accuracy: 0.6696 - val_loss: 0.5594 - val_accuracy: 0.6905
Epoch 8/20
6/6 [=================

In [92]:
nn_glove_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
nn_glove_test_acc

0.88

In [ ]:
# Create classifier summary table

models = ['Linear SVC', 'Random Forest', 'Multinomial Naive Bayes', 
                  'AdaBoost', 'XGBoost', 'Neural Network LSTM']

glove_model_accs = [svm_glove_test_acc, rf_glove_test_acc, mnb_glove_test_acc, 
                    ada_glove_test_acc, xgb_glove_test_acc, nn_glove_test_acc]

glove_model_summary = pd.DataFrame([models, glove_model_accs]).T

glove_model_summary.columns = ['Classifier', 'GloVe Accuracy - 100d']

glove_model_summary

,Classifier,GloVe Accuracy - 100d
0,Linear SVC,0.89
1,Random Forest,0.9
2,Multinomial Naive Bayes,0.64
3,AdaBoost,0.82
4,XGBoost,0.87
5,Neural Network LSTM,0.63
